In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from datetime import datetime, timedelta
from sentinelhub import (
    SHConfig,
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)
import numpy as np
from typing import List, Tuple, Union

from fetch_images import FetchImages
from eval_scripts import TRUE_IMAGE_EVAL_SCRIPT

load_dotenv()

C:\Users\Jakub Lechowski\Desktop\Programowanie\Hackatoon\Helcomapi\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

# Load data

In [7]:
data_oil = pd.read_csv(os.getenv("CSV_PATH"))
data_oil['Date_standard'] = pd.to_datetime(data_oil['Date_standard'])
data_oil.head(10)

,OBJECTID,HELCOM_ID,Spill_ID,Latitude,Longitude,Length__km,Width__km_,Area__km2_,EstimVol_m3,Date_standard,Polluter
0,6051,5787,GE/S-3,55.198333,14.065000,4.000,2.700,6.48000,0.427680,2023-06-20,SHIP
1,5697,5699,22UTSLÄPP0187,56.338611,17.351389,0.000,0.000,5.20000,0.031200,2022-10-23,SHIP
2,5576,5578,DK-22UTSLÄPP0181,57.672222,10.619167,0.000,0.000,0.04500,0.069440,2022-10-08,SHIP
3,5591,5593,FI-3,60.150700,26.522800,0.982,0.215,0.00000,0.050000,2022-08-02,SHIP
4,5669,5671,22UTSLÄPP0095,56.105000,15.483611,0.000,0.000,0.00100,0.000823,2022-06-28,SHIP
5,5663,5665,22UTSLÄPP0084,58.625833,16.799444,0.000,0.000,0.01500,0.001620,2022-06-14,SHIP
6,5622,5624,PL-16,55.172500,18.882700,0.300,0.030,0.00720,0.002000,2022-06-07,SHIP
7,5658,5660,22UTSLÄPP0071,55.418611,13.807222,0.000,0.000,0.01350,0.003812,2022-05-18,SHIP
8,5655,5657,22UTSLÄPP0067,59.440000,18.360278,0.000,0.000,0.00345,0.004289,2022-05-16,SHIP
9,5654,5656,22UTSLÄPP0068,55.635000,13.038056,0.000,0.000,0.00100,0.001358,2022-05-16,SHIP


In [3]:
data_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   OBJECTID       10 non-null     int64         
 1   HELCOM_ID      10 non-null     int64         
 2   Spill_ID       10 non-null     object        
 3   Latitude       10 non-null     float64       
 4   Longitude      10 non-null     float64       
 5   Length__km     10 non-null     float64       
 6   Width__km_     10 non-null     float64       
 7   Area__km2_     10 non-null     float64       
 8   EstimVol_m3    10 non-null     float64       
 9   Date_standard  10 non-null     datetime64[ns]
 10  Polluter       10 non-null     object        
dtypes: datetime64[ns](1), float64(6), int64(2), object(2)
memory usage: 1008.0+ bytes


# Fetch

### config

In [8]:
config = SHConfig()
config.sh_client_id = os.getenv("CLIENT_ID")
config.sh_client_secret = os.getenv("CLIENT_SECRET")
config.sh_token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
config.sh_base_url = "https://sh.dataspace.copernicus.eu"
config.save("cdse")

In [14]:
clinet = FetchImages(
    data=data_oil,
    config=config,
    evalscript=TRUE_IMAGE_EVAL_SCRIPT,
    size_km=100,
    threads=5,
    resolution=100,
    time_interval_size=10,
    verbose=True,
).fetch()

To see actual bbox go to this page http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000 and paste cords
(13.275757157207519, 54.74788288288285, 14.85424284279248, 55.648783783783756)
To see actual bbox go to this page http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000 and paste cords
(16.53871753135735, 55.88816066064955, 18.164060246442652, 56.789061561550454)
To see actual bbox go to this page http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000 and paste cords
(9.776828613217448, 57.221771771749545, 11.461504720182552, 58.12267267265045)
To see actual bbox go to this page http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000 and paste cords
(25.617772978500724, 59.70024954954955, 27.427827021499276, 60.601150450450454)
To see actual bbox go to this page http://bboxfinder.com/#0.000000,0.000000,0.000000,0.000000 and paste cords
(14.6758784195935, 55.65454954954954, 16.291343802606498, 56.55545045045045)
To see actual bbox go to this page http://bboxfinder.com

In [ ]:
clinet.rename_request_png_based_on_json()